In [ ]:
pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached datasets-2.7.1-py3-none-any.whl (451 kB)
     |████████████████████████████████| 191 kB 6.9 MB/s 
     |████████████████████████████████| 7.6 MB 44.2 MB/s 
     |████████████████████████████████| 182 kB 67.0 MB/s 
     |████████████████████████████████| 132 kB 57.3 MB/s 
     |████████████████████████████████| 212 kB 78.1 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!nvidia-smi

Tue Dec 13 05:45:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

抓dataset

In [ ]:
from datasets import list_datasets, load_dataset
from pprint import pprint

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "mrpc")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

dataset裡分成三個 train validation test

In [ ]:
print(dataset)
pprint(dataset['train'][1])

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
{'idx': 1,
 'label': 0,
 'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway "
              'in 1998 for $ 2.5 billion .',
 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold "
              'it to Safeway for $ 1.8 billion in 1998 .'}


In [ ]:
train_data=dataset['train']
test_data=dataset['test']
eval_data=dataset['validation']

In [ ]:
import json
from pprint import pprint

pprint(train_data[0]['sentence1'])

('Amrozi accused his brother , whom he called " the witness " , of '
 'deliberately distorting his evidence .')


In [ ]:
def read_data(dataset):
    #open file
    sentence1=[]
    sentence2=[]
    label=[]
    idx=[]
    for text in dataset:
        sentence1.append(text['sentence1'])
        sentence2.append(text['sentence2'])
        label.append(text['label'])
        idx.append(text['idx'])
    return sentence1,sentence2,label,idx

In [ ]:
train_sen1,train_sen2,trainlabel,trainidx=read_data(train_data)
trainlabel=train_data['label']
print(len(train_sen1))
print(train_sen1[0])

3668
Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .


In [ ]:
test_sen1,test_sen2,testlabel,testidx=read_data(test_data)
print(len(test_sen1))
print(test_sen1[0])

1725
PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .


In [ ]:
eval_sen1,eval_sen2,evallabel,evalidx=read_data(eval_data)
evalabel=eval_data['label']
print(len(eval_sen1))
print(eval_sen1[0])

408
He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .


In [ ]:
print('train 資料集數量= ',len(train_data['label']))
print('eval 資料集數量= ',len(eval_data['label']))
print('test 資料集數量= ',len(test_data['label']))

train 資料集數量=  3668
eval 資料集數量=  408
test 資料集數量=  1725


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from transformers import AutoTokenizer,BertTokenizerFast, BertTokenizer, AdamW, BertForQuestionAnswering
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer_fast = BertTokenizerFast.from_pretrained("bert-base-uncased")
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:


print(tokenizer.decode(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0])
print("token_type_ids\n", train_encodings['token_type_ids'][0])
print("attention_mask\n", train_encodings['attention_mask'][0])

print(len(train_encodings['input_ids'][0]))
print(len(train_encodings['token_type_ids'][0]))

[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

加入label

In [ ]:
def add_targets(encodings,label):
    encodings.update({'label':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [ ]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['label']))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
[unused0] [PAD] [unused0] [PAD] [unused0] [unused0] [PAD] [unused0] [PAD] [PAD] [PAD] [PAD] [unused0] [unused0] [PAD] [PAD] [PAD] [unused0] [PAD] [unused0] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [PAD] [PAD] [unused0] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [unused0] [PAD] [unused0] [PAD] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [PAD] [unused0] [unused0] [PAD] [PAD] [PAD] [unused0] [PAD] [PAD] [unused0] [unused0] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [PAD] [unused0] [PAD] [unused0] [unused0] [unused0] [unused0] [unused0] [PAD] [PAD] [unused0] [PAD] [PAD] [unused0] [PAD] [PAD] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [PAD] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [PAD] [PAD] [unused0] [PAD] [unused0] [PAD] [unused0] [unused0] [unused0] [unuse

定義dataset 並轉換成tensor格式


In [ ]:

from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
          2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
          3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
          1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
          2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

載入模型架構

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2)  #num_labels 設定類別數
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity",config=config)    

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

該來訓練模型囉

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 12      # 設定 training batch size 
eval_batch_size = 12      # 設定 eval batch size
num_train_epochs = 10      # 設定 epoch 

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 3060


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

eval_dataloader
metric = load_metric("accuracy")

<ipython-input-32-570f2d83a7b4>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


真正開始訓練

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = '/content/drive/Shareddrives'  # your folder


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    #loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: 把if刪掉了
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 2.760838747024536}
{'epoch': 0, 'step': 50, 'loss': 0.4806138575077057}
{'epoch': 0, 'step': 100, 'loss': 0.578758180141449}
{'epoch': 0, 'step': 150, 'loss': 0.6558881402015686}
{'epoch': 0, 'step': 200, 'loss': 0.4794261157512665}
{'epoch': 0, 'step': 250, 'loss': 0.5912277698516846}
{'epoch': 0, 'step': 300, 'loss': 0.6691401600837708}


eval Iteration:   0%|          | 0/34 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

...

{'epoch': 8, 'step': 0, 'loss': 0.0007955262553878129}
{'epoch': 8, 'step': 50, 'loss': 0.00025373455719091}
{'epoch': 8, 'step': 100, 'loss': 0.0005348023260012269}
{'epoch': 8, 'step': 150, 'loss': 0.0002510203921701759}
{'epoch': 8, 'step': 200, 'loss': 0.0026441721711307764}
{'epoch': 8, 'step': 250, 'loss': 0.00030710044666193426}
{'epoch': 8, 'step': 300, 'loss': 0.03824218362569809}


eval Iteration:   0%|          | 0/34 [00:00<?, ?it/s]

Iteration:   0%|          | 0/306 [00:00<?, ?it/s]

{'epoch': 9, 'step': 0, 'loss': 0.0004154003399889916}
{'epoch': 9, 'step': 50, 'loss': 0.04019869863986969}
{'epoch': 9, 'step': 100, 'loss': 0.00039690177072770894}
{'epoch': 9, 'step': 150, 'loss': 0.0004630202893167734}
{'epoch': 9, 'step': 200, 'loss': 0.000338268670020625}
{'epoch': 9, 'step': 250, 'loss': 0.000622972846031189}
{'epoch': 9, 'step': 300, 'loss': 0.00042430788744241}


eval Iteration:   0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
print(best_epoch)

{'epoch': 0, 'acc': 0.6813725490196079}


inference

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

In [ ]:
#cd drive

In [ ]:
#ls

In [ ]:
config = BertConfig.from_pretrained("./drive/Shareddrives/epoch_0/config.json") 
model = BertForSequenceClassification.from_pretrained("./drive/Shareddrives/epoch_0/pytorch_model.bin", config = config).to(device)

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification

In [ ]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

In [ ]:
sen1="lisa goes to school everyday"
sen2="lisa never goes to school"
#sen1="lisa is a singer"
#sen2="lisa is not a singer"

predict = mrpc_model(model, sen1, sen2)
print("sentence= : ", sen1)
print("sentence= : ", sen2)

print("predict_label : ", predict.item())
if predict.item():
  print("equivalent")
else:
  print("not_equivalent")


sentence= :  lisa goes to school everyday
sentence= :  lisa never goes to school
predict_label :  1
equivalent


In [ ]:
"""cnt=0
errorcnt=0
for i in range(len(eval_data['label'])):
    if eval_data['label'][i]==0 or eval_data['label'][i]==1:
        cnt+=1
        sen1=eval_data['sentence1'][i]
        sen2=eval_data['sentence2'][i]
        predict=mrpc_model(model,sen1,sen2)
        #print("sentence= : ", sen1)
        #print("sentence= : ", sen2)

        #print("predict_label : ", predict.item())
        if predict.item()!=eval_data['label'][i]:
            errorcnt+=1
        
print(cnt)
print(errorcnt)"""

408
130


存進google drive

In [ ]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

In [ ]:
#cd gdrive/MyDrive/Colab Notebooks

In [ ]:
#torch.save(model,"./test_model2")